In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/ML/kobert

Mounted at /content/drive
/content/drive/My Drive/ML/kobert


In [2]:
!pip install pytorch-transformers

     |████████████████████████████████| 176 kB 5.3 MB/s 
     |████████████████████████████████| 131 kB 44.6 MB/s 
     |████████████████████████████████| 1.2 MB 48.2 MB/s 
     |████████████████████████████████| 895 kB 42.8 MB/s 
     |████████████████████████████████| 79 kB 6.7 MB/s 
     |████████████████████████████████| 8.5 MB 36.1 MB/s 
     |████████████████████████████████| 138 kB 42.2 MB/s 
     |████████████████████████████████| 127 kB 43.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig

In [4]:
train_df =  pd.read_csv('data/train.tsv', delimiter='\t')
test_df =  pd.read_csv('data/test.tsv', delimiter='\t')

In [5]:
train_df.isnull().sum()
max(train_df['comments'].apply(lambda x: len(str(x))))

135

In [6]:
# 일단 안쓰는 genderbias하고 bias 열 삭제
train_df.drop(['contain_gender_bias', 'bias'], axis=1, inplace=True)
test_df.drop(['contain_gender_bias', 'bias'], axis=1, inplace=True)
# hate/offensive
# 원래 3종류 분륜데, 이진 분류 모델에 맞춰서 2개로 일단 분류함
#train_df['hate'] = train_df['hate'].map({'none':0, 'offensive':1, 'hate':2})
train_df['hate'] = train_df['hate'].map({'none':0, 'offensive':1, 'hate':1})
train_df.head(5)

,comments,hate
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,1
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,0
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",1
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",0
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,1


In [7]:
class HateDataSet(torch.utils.data.Dataset): 
  def __init__(self, df):
    self.df = df
  def __len__(self):
    return len(self.df)
  def __getitem__(self, idx): 
    text = self.df.iloc[idx, 0]
    label = self.df.iloc[idx, 1]
    return text, label

In [8]:
train_dataset = HateDataSet(train_df)
train_loader = DataLoader(train_dataset, batch_size=2, num_workers=2, shuffle=True)

In [9]:
## CPU
# device = torch.device("cpu")

## GPU
device = torch.device("cuda:0")

In [10]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased')
model.to(device)

100%|██████████| 714314041/714314041 [00:25<00:00, 28162431.41B/s]


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [11]:
learning_rate = 1e-6
epochs = 5

optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
itr = 1
p_itr = 500
epochs = 10
total_loss = 0
total_len = 0
total_correct = 0


model.train()
for epoch in range(epochs):
    
    for text, label in train_loader:
        optimizer.zero_grad()
        # encoding and zero padding
        encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
        padded_list =  [e + [0] * (256-len(e)) for e in encoded_list]


        sample = torch.tensor(padded_list)
        sample, label = sample.to(device), label.to(device)
        labels = torch.tensor(label)
        outputs = model(sample, labels=labels)
        loss, logits = outputs

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0

        itr+=1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[Epoch 1/10] Iteration 500 -> Train Loss: 0.5905, Accuracy: 0.686
[Epoch 1/10] Iteration 1000 -> Train Loss: 0.5953, Accuracy: 0.665
[Epoch 1/10] Iteration 1500 -> Train Loss: 0.5875, Accuracy: 0.689
[Epoch 1/10] Iteration 2000 -> Train Loss: 0.5678, Accuracy: 0.709
[Epoch 1/10] Iteration 2500 -> Train Loss: 0.5994, Accuracy: 0.685
[Epoch 1/10] Iteration 3000 -> Train Loss: 0.5680, Accuracy: 0.695
[Epoch 1/10] Iteration 3500 -> Train Loss: 0.5686, Accuracy: 0.706
[Epoch 2/10] Iteration 4000 -> Train Loss: 0.5525, Accuracy: 0.711
[Epoch 2/10] Iteration 4500 -> Train Loss: 0.5355, Accuracy: 0.726
[Epoch 2/10] Iteration 5000 -> Train Loss: 0.5534, Accuracy: 0.721
[Epoch 2/10] Iteration 5500 -> Train Loss: 0.5359, Accuracy: 0.749
[Epoch 2/10] Iteration 6000 -> Train Loss: 0.5480, Accuracy: 0.719
[Epoch 2/10] Iteration 6500 -> Train Loss: 0.5619, Accuracy: 0.694
